In [55]:
import xarray
import hvplot.xarray
from dask.distributed import Client
import numpy as np
import pandas as pd
import datetime
import panel as pn
pn.extension()

In [2]:
infile = r'C:\Users\xavier.mouy\Documents\PhD\Projects\Dectector\DFO_RCA_run\test_dataset\summary\hourly_summary.nc'

In [79]:
global ds
ds = xarray.open_dataset(infile)
ds

<xarray.Dataset>
Dimensions:     (confidence: 10, date: 104)
Coordinates:
  * date        (date) datetime64[ns] 2019-04-10T15:00:00 ... 2019-04-14T22:0...
  * confidence  (confidence) float64 0.5 0.55 0.6 0.65 0.7 ... 0.8 0.85 0.9 0.95
Data variables:
    FS          (confidence, date) float64 ...
    NN          (confidence, date) float64 ...

In [97]:
# widgets
class_list = list(ds.data_vars)
class_name  = pn.widgets.Select(name='Class',
                                value=class_list[0],
                                options=class_list
                               )
threshold = pn.widgets.DiscreteSlider(name='Confidence', options=list(ds.confidence.data), value=list(ds.confidence.data)[0])

file_input = pn.widgets.FileInput()
file_selector = pn.widgets.FileSelector('~')

In [80]:
@pn.depends(class_name,threshold)
def plot1D(class_name,threshold,time_resolution='H'):
    global ds
    if time_resolution == 'H':
        plot = ds[class_name].sel(confidence=threshold,method='nearest').hvplot()
    elif time_resolution == 'D':
        plot = ds[class_name].sel(confidence=threshold,method='nearest').resample(date='D').sum().hvplot(kind='step', width=820)
    else:
        print('Time resolution not supported. Use "H" for hours, or "D" for days.')
    return plot

In [83]:
@pn.depends(class_name,threshold)
def plot2D(class_name,threshold):
    global ds
    ds1 = ds[class_name].sel(confidence=threshold,method='nearest')
    # Date vector
    date = list(ds1.resample(date='D').groups.keys())
    # hour of day vector
    #hour = pd.date_range("00:00", freq="1H", periods=24).time
    hour = range(0,24,1)

    # detections matrix
    detections = np.empty((len(hour),len(date),))
    detections[:] = np.nan
    detec_matrix = xarray.DataArray(detections, coords={'hour': hour, 'date': date}, dims=['hour','date'])

    # Fill in 2D Array
    for x in ds1:
        fulldate = pd.to_datetime(x.date.data)
        day_str = datetime.datetime.strftime(fulldate,'%Y-%m-%d')
        hour = fulldate.time().hour
        nDetec = x.data
        detec_matrix.loc[hour,day_str] = nDetec

    # Sum of detec per day
    detec_sum = detec_matrix.sum(dim='hour')
    return detec_matrix.hvplot(cmap='rainbow',width=800)

In [98]:
widgets = pn.Column(file_selector, class_name,threshold)
plots = pn.Column(plot2D,plot1D)
app = pn.Row(widgets,plots)
app.show()
#widgets

Launching server at http://localhost:61826


In [42]:
ds

<xarray.Dataset>
Dimensions:     (confidence: 10, date: 104)
Coordinates:
  * date        (date) datetime64[ns] 2019-04-10T15:00:00 ... 2019-04-14T22:0...
  * confidence  (confidence) float64 0.5 0.55 0.6 0.65 0.7 ... 0.8 0.85 0.9 0.95
Data variables:
    FS          (confidence, date) float64 262.0 568.0 451.0 ... 0.0 0.0 0.0
    NN          (confidence, date) float64 1.056e+03 1.657e+03 ... 497.0

In [50]:
list(ds.confidence.data)

[0.5,
 0.55,
 0.6000000000000001,
 0.6500000000000001,
 0.7000000000000002,
 0.7500000000000002,
 0.8000000000000003,
 0.8500000000000003,
 0.9000000000000004,
 0.9500000000000004]

In [51]:
list(ds.confidence.data)[0]

0.5

In [93]:
file_input.filename


'hourly_summary.nc'

In [95]:
ds3 = xarray.open_dataset(file_input.value)

ValueError: can't open netCDF4/HDF5 as bytes try passing a path or file-like object